## Georelation explained

In [2]:
import lxml.etree as etree
import requests
import cioppy

import numpy as np
import ipyleaflet
from shapely.geometry import box
from shapely.geometry import multipolygon
from shapely.wkt import loads
from ipyleaflet import Map, Polygon

ciop = cioppy.Cioppy()

In [3]:
def get_params(osd):

    oss_ns = {'a':'http://www.w3.org/2001/XMLSchema', 
          'b':'http://www.w3.org/2001/XMLSchema-instance',
          'c':'http://a9.com/-/opensearch/extensions/time/1.0/',
          'd':'http://www.opengis.net/eop/2.0',
          'e':'http://purl.org/dc/terms/',
          'f':'http://a9.com/-/spec/opensearch/extensions/parameters/1.0/',
          'g':'http://purl.org/dc/elements/1.1/',
          'h':'http://www.terradue.com/opensearch',
          'i':'http://a9.com/-/opensearch/extensions/geo/1.0/',
          'j':'http://a9.com/-/spec/opensearch/1.1/'}
    
    oss_content = etree.fromstring(requests.get(osd).content)
    
    url_template_element = oss_content.xpath('/j:OpenSearchDescription/j:Url[@type="application/atom+xml"]',
                                                 namespaces=oss_ns)[0]
    
    parameters = dict()
    
    for index, parameter in enumerate(url_template_element.xpath('.//f:Parameter', namespaces=oss_ns)):
        
        parameters[parameter.attrib['name']] = {'title' : parameter.attrib['title'], 
                                                'value' : parameter.attrib['value']}
        
        options = []
        for option in parameter.xpath('.//f:Option', namespaces=oss_ns):
            
            options.append(option.attrib['value'])
    
        parameters[parameter.attrib['name']] = {'title' : parameter.attrib['title'], 
                                                'value' : parameter.attrib['value'],
                                                'options' : options}
    return parameters

In [4]:
def get_param_value(osd, os_parameter):

    params = get_params(osd)

    res = None
    
    for index, param in enumerate(params):
        
        if params[param]['value'] == os_parameter:
            
            res = params[param]
            res['name'] = param
     
    return res

Define the Sentinel-1 endpoint

In [5]:
s1_osd_url = 'https://catalog.terradue.com/sentinel1/description'

Get the Sentinel-1 search parameters

In [6]:
s1_parameters = get_params(s1_osd_url)

Print all the OpenSearch parameters

In [7]:
for key, value in s1_parameters.iteritems():

    print '%24s' % s1_parameters[key]['value'],  s1_parameters[key]['title'] 

              {geo:uid?} The identifier of the resource within the search engine context (local reference)
    {t2:downloadOrigin?} a string that identifies the download origin (keyword, hostname...) to adapt the enclosure. If the parameter is enclosed between [] (e.g. [terradue]), enclosure will be returned only if there is a enclosure found for this source.
           {startIndex?} index of the first search result desired
            {startPage?} page number of the set of search results desired
       {eop:sensorType?} A string identifying the sensor type
     {eop:accessedFrom?} A string identifying the location from which the resource will be accessed. The catalogue shall return the download location in the enclosure atom link according to the parameter value.
         {t2:landCover?} A number, set or interval requesting the land coverage
      {eop:productType?} A string identifying the product type
            {eop:title?} A name given to the resource
           {time:start?} sta

Find the parameter associated to **'{eop:productType?}'** and its options

In [8]:
geo_relation_parameter = get_param_value(s1_osd_url, '{geo:relation?}')

In [9]:
geo_relation_parameter['name']

'rel'

In [10]:
options = s1_parameters[geo_relation_parameter['name']]['options']

In [11]:
options

['intersects', 'contains', 'disjoint']

Define an area of interest

In [12]:
aoi_wkt = 'POLYGON ((20.9113 39.4866, 20.9113 40.0866, 20.3113 40.0866, 20.3113 39.4866, 20.9113 39.4866))'

Find the **{geo:geometry?}** parameter name

In [13]:
wkt_parameter = get_param_value(s1_osd_url, '{geo:geometry?}')

In [15]:
wkt_parameter

{'name': 'geom',
 'options': [],
 'title': 'Geometry in WKT',
 'value': '{geo:geometry?}'}

### Relation intersects

In [16]:
search_params = dict([(geo_relation_parameter['name'], 
                       options[0]),
                    (wkt_parameter['name'], aoi_wkt)])

In [17]:
search_params

{'geom': 'POLYGON ((20.9113 39.4866, 20.9113 40.0866, 20.3113 40.0866, 20.3113 39.4866, 20.9113 39.4866))',
 'rel': 'intersects'}

In [31]:
search = ciop.search(end_point=s1_osd_url,
                     params=search_params,
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                     model='EOP')

Show the first result returned

In [41]:
for result in search:
    print result['identifier']

S1B_IW_RAW__0SDV_20181021T162349_20181021T162421_013253_018804_96CD
S1B_IW_SLC__1SDV_20181021T162351_20181021T162418_013253_018804_7F52
S1B_IW_GRDH_1SDV_20181021T162352_20181021T162417_013253_018804_B781
S1B_IW_RAW__0SDV_20181021T162324_20181021T162356_013253_018804_E4EF
S1B_IW_SLC__1SDV_20181021T162326_20181021T162354_013253_018804_4BBD
S1B_IW_GRDH_1SDV_20181021T162327_20181021T162352_013253_018804_3E9D
S1B_IW_OCN__2SDV_20181021T162327_20181021T162352_013253_018804_2A11
S1B_IW_RAW__0SDV_20181021T162259_20181021T162331_013253_018804_D713
S1A_IW_RAW__0SDV_20181021T043148_20181021T043221_024229_02A69F_F9F8
S1A_IW_SLC__1SDV_20181021T043151_20181021T043218_024229_02A69F_122B
S1A_IW_OCN__2SDV_20181021T043152_20181021T043217_024229_02A69F_5AAE
S1A_IW_GRDH_1SDV_20181021T043152_20181021T043217_024229_02A69F_C169
S1A_IW_SLC__1SDV_20181021T043126_20181021T043153_024229_02A69F_FFAB
S1A_IW_OCN__2SDV_20181021T043127_20181021T043152_024229_02A69F_7132
S1A_IW_GRDH_1SDV_20181021T043127_20181021T043152

### Relation contains

In [26]:
buffer_size = 4

extended_aoi_wkt = box(*loads(aoi_wkt).buffer(buffer_size).bounds).wkt


In [34]:
search_params = dict([(geo_relation_parameter['name'], 
                       options[1]),
                     (wkt_parameter['name'], extended_aoi_wkt)])

In [35]:
search_params

{'geom': 'POLYGON ((24.9113 35.4866, 24.9113 44.0866, 16.3113 44.0866, 16.3113 35.4866, 24.9113 35.4866))',
 'rel': 'contains'}

In [36]:
search = ciop.search(end_point=s1_osd_url,
                     params=search_params,
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                     model='EOP')

### Relation disjoint

In [42]:
buffer_size = 15

extended_aoi_wkt = box(*loads(aoi_wkt).buffer(buffer_size).bounds).wkt


In [43]:
search_params = dict([(geo_relation_parameter['name'], 
                       options[2]),
                     (wkt_parameter['name'], extended_aoi_wkt)])

In [44]:
search_params

{'geom': 'POLYGON ((35.9113 24.4866, 35.9113 55.0866, 5.311299999999999 55.0866, 5.311299999999999 24.4866, 35.9113 24.4866))',
 'rel': 'disjoint'}

In [46]:
search = ciop.search(end_point=s1_osd_url,
                     params=search_params,
                     output_fields='self,productType,track,enclosure,identifier,wkt,startdate', 
                     model='EOP')

In [47]:
for result in search:
    print result['identifier']

S1A_EW_RAW__0SDV_20181023T073503_20181023T073606_024260_02A7A5_D52C
S1A_EW_RAW__0SDV_20181023T073403_20181023T073512_024260_02A7A5_B65C
S1A_EW_RAW__0SDV_20181023T073303_20181023T073412_024260_02A7A5_0A53
S1A_EW_RAW__0SDV_20181023T073203_20181023T073312_024260_02A7A5_B5B2
S1A_EW_RAW__0SDH_20181023T072540_20181023T072619_024260_02A7A4_7F54
S1A_EW_RAW__0SDH_20181023T072440_20181023T072548_024260_02A7A4_1451
S1A_EW_RAW__0SDH_20181023T072340_20181023T072448_024260_02A7A4_33AB
S1A_EW_RAW__0SDH_20181023T072240_20181023T072348_024260_02A7A4_50D5
S1A_EW_RAW__0SDH_20181023T072140_20181023T072248_024260_02A7A4_57B4
S1A_EW_RAW__0SDH_20181023T072040_20181023T072148_024260_02A7A4_FE27
S1A_IW_RAW__0SDV_20181023T070843_20181023T070914_024260_02A7A1_8403
S1A_IW_RAW__0SDV_20181023T070818_20181023T070850_024260_02A7A1_244B
S1B_IW_RAW__0SDV_20181023T064709_20181023T064736_013276_0188B3_8D4E
S1B_IW_RAW__0SDV_20181023T064644_20181023T064716_013276_0188B3_6784
S1B_IW_RAW__0SDV_20181023T064619_20181023T064651